In [ ]:
'''
Citations :
[1] “Beautiful Soup documentation,” Beautiful Soup 4.4.0 documentation. [Online]. Available: https://beautiful-soup-4.readthedocs.io/en/latest/.
[2] "Selenium." [Online]. Available: https://www.selenium.dev/. 
'''

In [ ]:
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common import exceptions 
import datetime
from html.parser import HTMLParser
from re import sub
from sys import stderr
from traceback import print_exc
import re
import numpy as np

In [ ]:
woLs = []
cmnts = []

In [ ]:
# for op in opiC.Generic_Name:
for op in pd.read_csv('final_opioid_keywords.csv').Generic_Name:
    url = "https://old.reddit.com/r/"+op+"/"
    page = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
    soup = BeautifulSoup(page.text, 'html.parser')
    chk_18 = soup.find_all("button",class_="c-btn c-btn-primary") or 'NONE'
    if chk_18 != 'NONE':
        driver = webdriver.Chrome("driver/chromedriver.exe")
        driver.get(url)
        driver.find_elements_by_name('over18')[1].click()
        counter = 1
        nr11 = driver.find_elements_by_css_selector('a.title.may-blank') or driver.find_elements_by_css_selector('a.title.may-blank.outbound') or ["NONE"]
        while counter<100:
            if nr11[0] != 'NONE':
                if counter == 1:
                    for nl1 in nr11:
                        if nl1 != 'NONE':
                            woLs.append([op,nl1.text])
                            c_L3 = driver.find_element_by_link_text(nl1.text)
                            page = requests.get(c_L3.get_attribute('href'))
                            soup = BeautifulSoup(page.text, 'html.parser')
                            chkCmnts = soup.find_all('div', class_="commentarea")
                            cmnts.append([op,chkCmnts]) 
                    counter = counter+1
                else:
                    try:
                        nL = driver.find_element_by_css_selector('.next-button [href]').get_attribute('href')
                        driver.get(nL)
                        nr12 = driver.find_elements_by_css_selector('a.title.may-blank') or driver.find_elements_by_css_selector('a.title.may-blank.outbound') or ["NONE"]
                        for nl2 in nr12:
                            if nl2 != 'NONE':
                                woLs.append([op,nl2.text])
                                c_L3 = driver.find_element_by_link_text(nl2.text)
                                page = requests.get(c_L3.get_attribute('href'), headers={'User-Agent': 'Mozilla/5.0'})
                                soup = BeautifulSoup(page.text, 'html.parser')
                                chkCmnts = soup.find_all('div', class_="commentarea")
                                cmnts.append([op,chkCmnts]) 
                        counter = counter+1
                    except:
                        break
            else:
                break
        driver.close()
    else:
        driver = webdriver.Chrome("driver/chromedriver.exe")
        driver.get(url)
        counter = 1
        nr11 = driver.find_elements_by_css_selector('a.title.may-blank') or driver.find_elements_by_css_selector('a.title.may-blank.outbound') or ["NONE"]
        while counter<100:
            if nr11[0] != "NONE":
                if counter == 1:
                    for nl1 in nr11:
                        if nl1 != 'NONE':
                            woLs.append([op,nl1.text])
                            c_L3 = driver.find_element_by_link_text(nl1.text)
                            print(c_L3.get_attribute('href'))
                            page = requests.get(c_L3.get_attribute('href'), headers={'User-Agent': 'Mozilla/5.0'})
                            soup = BeautifulSoup(page.text, 'html.parser')
                            chkCmnts = soup.find_all('div', class_="commentarea")
                            cmnts.append([op,chkCmnts])
                    counter = counter+1
                else:
                    try:
                        nL = driver.find_element_by_css_selector('.next-button [href]').get_attribute('href')
                        driver.get(nL)
                        nr12 = driver.find_elements_by_css_selector('a.title.may-blank') or driver.find_elements_by_css_selector('a.title.may-blank.outbound') or ["NONE"]
                        for nl2 in nr12:
                            if nl2 != 'NONE':
                                woLs.append([op,nl2.text])
                                c_L3 = driver.find_element_by_link_text(nl2.text)
                                page = requests.get(c_L3.get_attribute('href'), headers={'User-Agent': 'Mozilla/5.0'})
                                soup = BeautifulSoup(page.text, 'html.parser')
                                chkCmnts = soup.find_all('div', class_="commentarea")
                                cmnts.append([op,chkCmnts]) 
                        counter = counter+1
                    except:
                        break
            else:
                break
        driver.close()

In [ ]:
FinRed = woLs+cmnts
FinRedOpi = pd.DataFrame(FinRed,columns=['Components','Comments'])
len(FinRedOpi['Comments'])

In [ ]:
twisS = []

In [ ]:
for twis in range(len(FinRedOpi)):
    print(twis) 
    htmlParse = BeautifulSoup(FinRedOpi.Comments[twis], 'html.parser')
    for jkl in htmlParse.find_all('div',class_='entry unvoted'):
        if jkl.find('div',class_='md') != None:
            twisS.append([jkl.find('div',class_='md').text, jkl.find('time')['datetime'],FinRedOpi.Components[twis]])

In [ ]:
newCmntsOpi = pd.DataFrame(twisS,columns=['Comments','DateTime','Components'])
newCmntsOpi = newCmntsOpi.dropna()
newCmntsOpi['Comments'] = [re.sub(r"\[.*\n?","",str(x),flags=re.MULTILINE) for x in newCmntsOpi['Comments']]
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.replace(r'<[^<>]*>', '', regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.split('\n\n')
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].replace({r'\s+$': '', r'^\s+': ''}, regex=True).replace(r'\n',  ' ', regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].replace(r'^\s*$', np.nan, regex=True)
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.rstrip()
newCmntsOpi['Comments'] = newCmntsOpi['Comments'].str.lstrip()
newCmntsOpi = newCmntsOpi.dropna()
newCmntsOpi = newCmntsOpi.reset_index(drop=True)
len(newCmntsOpi)

In [ ]:
newCmntsOpi.to_csv('webdata/reddit_effects.csv')
newCmntsOpi